In [461]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestClassifier

%matplotlib inline

In [489]:
df = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [490]:
df['Title'] = df['Name'].str.extract(r', (\w+\.)')
df['Name_Len'] = df['Name'].apply(len)

df['Age_NA'] = df['Age'].isna().astype('int16')
df['Age'].fillna(df['Age'].mean(), inplace = True)

df['Fare'].fillna(df['Fare'].mean(), inplace = True)

tickets = df.Ticket.str.split(" ", 1)

df['Ticket_Num'] = tickets.apply(lambda x: x.pop()).str.split(" ", 1).apply(lambda x: x[-1]).str.replace('LINE', "-1").astype('int32')

df['Ticket_Pre'] = tickets.apply(lambda x: x[0] if len(x)>0 else 'None').str.replace('.', '').str.replace('/', '').str.upper()

df['Deck'] = df['Cabin'].astype(str).str[0]

C:\Users\danbu\anaconda3\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  del sys.path[0]


In [491]:
scaler = StandardScaler()
ohe = OneHotEncoder(handle_unknown='ignore',sparse=False)
ordinal = OrdinalEncoder()

df['Sex'] = ordinal.fit_transform(df[['Sex']])

ohe_df = pd.DataFrame(ohe.fit_transform(df[['Cabin', 'Title', 'Embarked', 'Ticket_Pre', 'Deck']].fillna("NaN")))
df = df.join(ohe_df, rsuffix='ohe_')
df = df.drop(columns=['Cabin', 'Title', 'Embarked', 'Ticket_Pre', 'Deck'])

df[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Name_Len', 'Ticket_Num']] = scaler.fit_transform(df[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Name_Len', 'Ticket_Num']])

In [492]:
df = df.drop('Name', axis=1)
df = df.drop('Ticket', axis=1)

# df = df.join(pd.get_dummies(df['Cabin'], dummy_na='Cabin_NA', prefix='Cabin')).drop('Cabin', axis=1)
# df = df.join(pd.get_dummies(df['Title'], dummy_na='Title_NA', prefix='Title')).drop('Title', axis=1)
# df = df.join(pd.get_dummies(df['Embarked'], dummy_na='Embarked_NA', prefix='Embarked')).drop('Embarked', axis=1)
# df = df.join(pd.get_dummies(df['Ticket_Pre'], dummy_na='Ticket_Pre_NA', prefix='Ticket_Pre')).drop('Ticket_Pre', axis=1)
# df = df.join(pd.get_dummies(df['Deck'], dummy_na='Deck_NA', prefix='Deck')).drop('Deck', axis=1)

In [493]:
df.head(10)

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Name_Len,Age_NA,...,197,198,199,200,201,202,203,204,205,206
0,1,0,0.827377,0.737695,-0.592481,0.432793,-0.473674,-0.502445,-0.427451,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,2,1,-1.566107,-1.355574,0.638789,0.432793,-0.473674,0.786845,2.590962,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,1,0.827377,-1.355574,-0.284663,-0.474545,-0.473674,-0.488854,-0.535252,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,4,1,-1.566107,-1.355574,0.407926,0.432793,-0.473674,0.420730,1.836359,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,0,0.827377,0.737695,0.407926,-0.474545,-0.473674,-0.486337,-0.319651,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5,6,0,0.827377,0.737695,0.000000,-0.474545,-0.473674,-0.478116,-1.182055,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
6,7,0,-1.566107,0.737695,1.870059,-0.474545,-0.473674,0.395814,-0.427451,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
7,8,0,0.827377,0.737695,-2.131568,2.247470,0.767630,-0.224083,0.327152,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
8,9,1,0.827377,-1.355574,-0.207709,-0.474545,2.008933,-0.424256,2.375361,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
9,10,1,-0.369365,-1.355574,-1.208115,0.432793,-0.473674,-0.042956,0.866154,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [494]:
X = df.drop(columns=['PassengerId', 'Survived'])
y = df['Survived']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=0)

In [495]:
# params = {'max_depth': np.arange(24, 40),
#          'min_samples_split': np.arange(2,6),
#          'min_samples_leaf': np.arange(1, 10)}

# clf = GridSearchCV(RandomForestClassifier(), params, verbose=2)
# clf.fit(X_train, y_train)

# print(clf.best_params_, clf.best_score_)

In [496]:
forest = RandomForestClassifier(n_estimators=500,
                                max_depth=25,
                                min_samples_split=5,
                                min_samples_leaf=1,
                                random_state=0,
                                n_jobs=-1)
forest.fit(X_train, y_train)

print(accuracy_score(y_test, forest.predict(X_test)))

0.8324022346368715


In [497]:
df = test.copy()


In [498]:
df['Title'] = df['Name'].str.extract(r', (\w+\.)')
df['Name_Len'] = df['Name'].apply(len)

df['Age_NA'] = df['Age'].isna().astype('int16')
df['Age'].fillna(df['Age'].mean(), inplace = True)

df['Fare'].fillna(df['Fare'].mean(), inplace = True)

tickets = df.Ticket.str.split(" ", 1)

df['Ticket_Num'] = tickets.apply(lambda x: x.pop()).str.split(" ", 1).apply(lambda x: x[-1]).str.replace('LINE', "-1").astype('int32')

df['Ticket_Pre'] = tickets.apply(lambda x: x[0] if len(x)>0 else 'None').str.replace('.', '').str.replace('/', '').str.upper()

df['Deck'] = df['Cabin'].astype(str).str[0]

df['Sex'] = ordinal.transform(df[['Sex']])

ohe_df = pd.DataFrame(ohe.transform(df[['Cabin', 'Title', 'Embarked', 'Ticket_Pre', 'Deck']].fillna("NaN")))
df = df.join(ohe_df, rsuffix='ohe_')
df = df.drop(columns=['Cabin', 'Title', 'Embarked', 'Ticket_Pre', 'Deck'])

df[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Name_Len', 'Ticket_Num']] = scaler.transform(df[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Name_Len', 'Ticket_Num']])


C:\Users\danbu\anaconda3\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  del sys.path[0]


In [503]:
X = df.drop(columns=['Name', 'Ticket'])

print(X.isna().sum()[:20])

res = pd.DataFrame(X['PassengerId'])

res['Survived'] = forest.predict(X.drop(columns=['PassengerId']))

PassengerId    0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Name_Len       0
Age_NA         0
Ticket_Num     0
0              0
1              0
2              0
3              0
4              0
5              0
6              0
7              0
8              0
9              0
dtype: int64


In [506]:
res.to_csv("submission.csv", index=False)